In [1]:
# Read in Poore and Nemecek 2018 data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import re

Poore And Nemecek (2018) [1] present a comprehensive list of LCA impact values for a series of food items:

- Land Use
- GHG Emissions (IPCC 2013)
- GHG Emissions (IPCC 2007)
- Acidifying Emissions
- Eutrophying Emissions
- Freshwater Withdrawals
- Stress-Weighted Water Use

The values are compiled from a metaanalysis of several global studies.
Regional values can be derived from the main LCA model spreadsheet which can be downloaded from here:
https://ora.ox.ac.uk/objects/uuid:a63fb28c-98f8-4313-add6-e9eca99320a5

Using the weights in column W, values can be derived for each region.

This notebook will read the values from an intermediate file with the summary values for each region and generate an xarray dataset which can be directly used with the `AgriFoodPy` package


In [2]:

datafile = "../../data/impact/Key for matching Impacts to Food Balance Sheets - Poore Nemecek 2018 Regional Impact data from Poore.xlsx"


In [3]:

data = pd.read_excel(datafile, header=(0,1), na_values=['-'], index_col=0)
# data = data.rename(columns={"Mean.1":"GHGE intensity global mean (kg CO2e/kg)"})
# data[['Product','GHGE intensity global mean (kg CO2e/kg)']]
data

Global (per kg or per liter)  \
Food                                        Land Use (m2)   
Beer                                                  1.1   
Wine                                                  1.8   
Tomatoes                                              0.8   
Onions and Leeks                                      0.4   
Root Vegetables                                       0.3   
Cabbages and Other Brassicas                          0.6   
Other Vegetables                                      0.4   
Citrus Fruit                                          0.9   
Bananas                                               1.9   
Apples                                                0.6   
Berries and Grapes                                    2.4   
Other Fruit                                           0.9   
Avocado                                               2.6   
Soymilk                                               0.7   
Milk                                                  9.0   
Almond milk                                           0.5   
Oat milk                                              0.8   
Rice milk                                             0.3   
Soybeans (Oil)                                       10.5   
Palm (Oil)                                            2.4   
Sunflower (Oil)                                      17.7   
Rapeseed (Oil)                                       10.6   
Olives (Oil)                                         26.3   
Beans & Pulses                                       15.6   
Peas                                                  7.5   
Nuts                                                 13.0   
Groundnuts                                            9.1   
Tofu                                                  3.5   
Bovine Meat (Beef Herd)                             326.2   
Bovine Meat (Dairy Herd)                             43.2   
Beef                                                202.3   
Mutton & Goat Meat                                  369.8   
Pig Meat                                             17.4   
Poultry Meat                                         12.2   
Cheese                                               87.8   
Eggs                                                  6.3   
Fish (farmed)                                         8.4   
Crustaceans (farmed)                                  3.0   
Bread                                                 3.9   
Maize Meal                                            2.9   
Oatmeal                                               7.6   
Rice                                                  2.8   
Potatoes                                              0.9   
Cassava and Other Roots                               1.8   
Coffee                                               21.6   
Tea                                                  28.1   
Chocolate                                            69.0   
Sugar (from sugar cane)                               2.0   
Sugar (from sugar beet)                               1.8   

                                                       \
Food                         GHG Emissions (kg CO2eq)   
Beer                                              1.2   
Wine                                              1.8   
Tomatoes                                          2.1   
Onions and Leeks                                  0.5   
Root Vegetables                                   0.4   
Cabbages and Other Brassicas                      0.5   
Other Vegetables                                  0.5   
Citrus Fruit                                      0.4   
Bananas                                           0.9   
Apples                                            0.4   
Berries and Grapes                                1.5   
Other Fruit                                       1.1   
Avocado                                           2.5   
Soymilk                                           1.0   
Milk 

In [4]:
# Stack the multicolumns into a multitiindex dataframe
df_stacked = data.stack(level=0).reset_index().rename(columns={"level_0": 'Item', 'level_1': 'Region'})
df_stacked_reindex = df_stacked.set_index(["Region", "Item"])
df_stacked_reindex.head()

,Food,Eutrophication (g PO43-eq),GHG Emissions (kg CO2eq),Land Use (m2),Water Use (L)
Region,Item,,,,
Asia (per kg or per liter produced),Beer,1.5,1.1,1.1,51.0
Europe (per kg or per liter produced),Beer,2.2,1.2,0.8,10.0
Global (per kg or per liter),Beer,2.3,1.2,1.1,17.0
L. America & Caribbean (per kg or per liter produced),Beer,1.7,1.1,1.5,10.4
North America (per kg or per liter produced),Beer,4.6,1.1,1.2,51.0


In [5]:
# Convert to xarray
data_xarray = df_stacked_reindex.to_xarray()

In [6]:
region_names = data_xarray.Region.values
region_names = [re.sub(r'\s*\([^)]*\)', '', region) for region in region_names]
data_xarray["Region"] = region_names

# Remove text in parentheses from variable names
variable_names = list(data_xarray.data_vars)
cleaned_variable_names = [re.sub(r'\s*\([^)]*\)', '', var) for var in variable_names]

# Create a dictionary to map old names to new names
name_mapping = dict(zip(variable_names, cleaned_variable_names))

# Rename the variables in the dataset
data_xarray = data_xarray.rename(name_mapping)
data_xarray

<xarray.Dataset>
Dimensions:         (Region: 6, Item: 49)
Coordinates:
  * Region          (Region) <U22 'Asia' 'Europe' ... 'North America' 'Oceania'
  * Item            (Item) object 'Almond milk' 'Apples' ... 'Tomatoes' 'Wine'
Data variables:
    Eutrophication  (Region, Item) float64 nan 2.0 nan 1.8 ... nan nan 1.1 nan
    GHG Emissions   (Region, Item) float64 nan 0.5 nan 0.8 ... nan nan 1.0 nan
    Land Use        (Region, Item) float64 nan 0.7 nan 1.3 ... nan nan 0.5 nan
    Water Use       (Region, Item) float64 nan 236.0 nan 11.0 ... nan 66.0 nan

In [7]:
data_xarray.to_netcdf("data/PN18_regional.nc",)